<a href="https://colab.research.google.com/github/Rociolivingstonux/Job-Application-Automation/blob/main/Demo_Automatizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Demo: Automatización de Postulaciones con Python
**Autora:** Rocío Livingston

Este cuaderno es una demostración interactiva de un script diseñado para automatizar el envío de correos personalizados (Cold Emailing).

**¿Cómo probarlo?**
1. Ejecuta la celda de abajo para instalar librerías.
2. Sube tus archivos de prueba (un Excel `contactos.xlsx` y un PDF de ejemplo).
3. Ejecuta el script principal e ingresa tus credenciales de prueba.



In [1]:
# 1. Instalamos las herramientas necesarias
!pip install pandas openpyxl

# 2. Importamos la función para subir archivos
from google.colab import files
import os

print("📂 PASO 1: Por favor, sube aquí tus archivos de prueba:")
print("   - Tu Excel (debe llamarse 'contactos.xlsx')")
print("   - Tu CV (ej: 'cv.pdf')")
print("   - Tu Carta (ej: 'carta.pdf')")
print("-" * 40)

# Esto abrirá una ventanita para subir archivos
uploaded = files.upload()

# Verificamos qué se subió
for fn in uploaded.keys():
  print(f'✅ Archivo cargado con éxito: "{fn}"')

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
import pandas as pd
import smtplib
import time
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from getpass import getpass

# --- CONFIGURACIÓN ---
# Aquí asumimos los nombres que subió el usuario
NOMBRE_EXCEL = "contactos.xlsx"
ASUNTO_BASE = "Postulación | Demo Python"

def limpiar_email(email):
    return re.sub(r'\s+', '', str(email))

def main():
    print("\n🔐 CONFIGURACIÓN DE ENVÍO")
    # Pedimos los nombres de los archivos PDF que subieron
    nombre_cv = input("Escribe el nombre exacto de tu archivo de CV (ej: cv.pdf): ")
    nombre_carta = input("Escribe el nombre de tu Carta (o deja vacío si no subiste): ")

    tu_email = input("Tu Email (Gmail): ").strip()
    tu_password = getpass("Tu Contraseña de Aplicación: ").strip()

    # Carga del Excel
    try:
        df = pd.read_excel(NOMBRE_EXCEL)
        df = df.fillna('')
        print(f"\n✅ Excel cargado: {len(df)} contactos encontrados.")
    except Exception as e:
        print(f"❌ Error: No se encontró '{NOMBRE_EXCEL}'. Por favor súbelo en el paso anterior.")
        return

    # Conexión SMTP
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(tu_email, tu_password)
        print("✅ Login exitoso. Iniciando simulación...")
    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return

    print("-" * 50)

    for index, row in df.iterrows():
        email_destino = limpiar_email(row['Email'])
        nombre = str(row['Nombre']).strip()
        empresa = str(row['Empresa']).strip()

        if "@" not in email_destino: continue

        # Personalización
        saludo = f"Hola {nombre}," if nombre else "Estimados,"
        texto_empresa = f"al equipo de {empresa}" if empresa else "a su equipo"
        asunto = f"{ASUNTO_BASE} - Para {empresa}" if empresa else ASUNTO_BASE

        # Cuerpo
        cuerpo = f"""
        {saludo}

        Esta es una prueba de envío automatizado para {texto_empresa}.
        Mi perfil combina Data Science con gestión administrativa.

        Adjunto la documentación correspondiente.

        Saludos,
        Bot de Prueba
        """

        msg = MIMEMultipart()
        msg['From'] = tu_email
        msg['To'] = email_destino
        msg['Subject'] = asunto
        msg.attach(MIMEText(cuerpo, 'plain'))

        # Adjuntar CV
        try:
            with open(nombre_cv, "rb") as f:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f"attachment; filename={nombre_cv}")
            msg.attach(part)
        except:
            print(f"⚠️ No se encontró el CV: {nombre_cv}")

        # Enviar
        try:
            server.send_message(msg)
            print(f"🚀 Enviado a: {email_destino}")
            time.sleep(5) # Pausa corta para demo
        except Exception as e:
            print(f"❌ Error: {e}")

    server.quit()
    print("🏁 Demo finalizada.")

if __name__ == "__main__":
    main()